In this notebook we'll explore the geospatial distribution of the bike bays in Leeds City Center. Unfortunately, the data is from 2016 but it's still fun to explore. We need some packages from folium to plot the data.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Import packages for geospatial analysis

import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
from shapely.geometry import Point

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Read in the data and have a look at it.

In [ ]:
bike_bays = pd.read_csv("../input/leeds-city-centre-bike-bays/bike-bays-in-leeds-city-centre.csv", encoding='latin-1')
bike_bays.head()

To center the map, we'll use the means of the bike bay locations:

In [ ]:
lat_mean = bike_bays['Lat '].mean()
long_mean = bike_bays['Long'].mean()

print(lat_mean, long_mean)

First, let's simply create a map with the locations of the bike bays:

In [ ]:
# create a map with the precise bike bay locations

# create a map focused around the mean of the bike bay locations
map_bay_locations = folium.Map(location=[lat_mean,long_mean], zoom_start=15)

for idx, row in bike_bays.iterrows():
    Marker([row['Lat '], row['Long']]).add_to(map_bay_locations)

map_bay_locations

That's nice. A heat map might give us some useful information. For example, it might suggest where new bike bays may be needed. Let's make one.

In [ ]:
# Create a heat map to suggest how the density of bike bays is spread

# Create a base map
bay_heat_map = folium.Map(location=[lat_mean,long_mean], zoom_start=15)

# Add a heatmap to the base map
HeatMap(data=bike_bays[['Lat ', 'Long']], radius=30).add_to(bay_heat_map)

# Display the map
bay_heat_map

Looks the station is a good place to park your bike!

Let's say that you're going out for lunch. Where should you park your bike? The following function takes in any location and outputs the closest bike bay. As an example, we use the location of the Corn Exchange.

To begin, we need to make a geometry column in our data which contain the locations of the bike bays as geometric points.

Then we define a function which takes in a location and outputs the closest bike bay,

In [ ]:
bike_bays_gdf = gpd.GeoDataFrame(bike_bays, geometry=gpd.points_from_xy(bike_bays['Long'], bike_bays['Lat ']))
bike_bays_gdf.crs = {'init': 'epsg:4326'}

def closest_bay(lunch_location):
    return bike_bays_gdf.iloc[bike_bays_gdf.geometry.distance(lunch_location).idxmin()]

Let's find out what the closest bike bay to the Corn Exchange is

In [ ]:
lunch_lat = 53.7960
lunch_long = -1.5402
lunch_name = "Leeds Corn Exchange"

lunch_location = Point(lunch_long,lunch_lat)
lunch_location.crs = {'init': 'epsg:4326'}

closest_bay(lunch_location)

And then plot the corn exchange and the cloest bike bay on a map

In [ ]:
# create a map focused around your lunch location
m_lunch = folium.Map(location=[lunch_lat,lunch_long], zoom_start=17)

Marker([closest_bay(lunch_location)['Lat '], closest_bay(lunch_location)['Long']],tooltip="Nearest bike bay").add_to(m_lunch)
Marker([lunch_lat, lunch_long],icon=folium.Icon(color='green'),tooltip=lunch_name).add_to(m_lunch)

m_lunch